# car ditection using YOLO

# import modul

In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

2024-06-06 22:15:45.354348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 22:15:45.354499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 22:15:45.357933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 22:15:45.374154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 22:15:46.957085: W tensorflow/compiler/tf2

In [2]:
# Filtering with a threshold on class scores
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=0.6):
    # Step 1: Compute box scores
    box_scores = box_confidence * box_class_probs

    # Step 2: Find the box_classes using the max box_scores, keep track of the corresponding score
    box_classes = tf.argmax(box_scores, axis=-1)
    box_class_scores = tf.reduce_max(box_scores, axis=-1)

    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold"
    filtering_mask = box_class_scores >= threshold

    # Step 4: Apply the mask to box_class_scores, boxes, and box_classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    filtered_boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)

    return scores, filtered_boxes, classes

In [3]:
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed=1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed=1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed=1)

In [4]:
# Apply the filter
scores, filtered_boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=0.6)

In [5]:
print("scores:", scores.numpy())
print("filtered_boxes:", filtered_boxes.numpy())
print("classes:", classes.numpy())

scores: [21.294851 62.658787 16.064978 ... 33.62834  54.689083 58.749218]
filtered_boxes: [[ 2.88381     2.9140406  -0.03477657  2.7024395 ]
 [ 4.01801     7.102514   -1.421094   -1.1415017 ]
 [ 1.0273384  -2.1178942   4.8872733  -4.0143332 ]
 ...
 [-0.27290797  7.283095    5.799567   -1.5821693 ]
 [-0.98508644 -2.4997838   7.1540446  -5.151011  ]
 [ 8.872453   -0.37440765  0.8893236   3.25239   ]]
classes: [32 73 78 ... 15 73 44]


# Non-max suppression

In [6]:
def iou(box1, box2):
   

    # Assign variable names to coordinates for clarity
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (xi1, yi1, xi2, yi2) coordinates of the intersection of box1 and box2
    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)
    
    # Calculate the width and height of the intersection
    inter_width = max(xi2 - xi1, 0)
    inter_height = max(yi2 - yi1, 0)
    
    # Calculate the area of the intersection
    inter_area = inter_width * inter_height
    
    # Calculate the area of both bounding boxes
    box1_area = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)
    box2_area = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)
    
    # Calculate the union area
    union_area = box1_area + box2_area - inter_area
    
    # Compute the IoU
    iou = inter_area / union_area
    
    return iou


In [7]:
## Test case 1: boxes intersect
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou for intersecting boxes = " + str(iou(box1, box2)))

## Test case 2: boxes do not intersect
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou for non-intersecting boxes = " + str(iou(box1,box2)))

## Test case 3: boxes intersect at vertices only
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou for boxes that only touch at vertices = " + str(iou(box1,box2)))

## Test case 4: boxes intersect at edge only
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))

iou for intersecting boxes = 0.14285714285714285
iou for non-intersecting boxes = 0.0
iou for boxes that only touch at vertices = 0.0
iou for boxes that only touch at edges = 0.0


In [8]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    
    
    # Ensure max_boxes is a tensor for tf.image.non_max_suppression
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_output_size=max_boxes, iou_threshold=iou_threshold)
    
    # Use tf.gather() to select only nms_indices from scores, boxes and classes
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)
    
    return scores, boxes, classes


In [9]:
scores = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
boxes =tf.random.normal([54, 4], mean=1, stddev=4, seed = 1)
classes =tf.random.normal([54,], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

scores[2] = 6.082263
boxes[2] = [ 4.110873   2.544206   2.1469054 -1.5198021]
classes[2] = 5.049652
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


# Wrapping up the filtering

In [10]:
def yolo_boxes_to_corners(box_xy, box_wh):
    """
    Convert YOLO box predictions to bounding box corners.
    Arguments:
    box_xy -- tensor of shape (None, 19, 19, 5, 2)
    box_wh -- tensor of shape (None, 19, 19, 5, 2)
    Returns:
    boxes -- tensor of shape (None, 19, 19, 5, 4)
    """
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return tf.concat([box_mins[..., 1:2], box_mins[..., 0:1], box_maxes[..., 1:2], box_maxes[..., 0:1]], axis=-1)

In [11]:
def scale_boxes(boxes, image_shape):
    """
    Scales the predicted boxes in order to be drawable on the image
    Arguments:
    boxes -- tensor of shape (None, 4) containing the coordinates of the boxes
    image_shape -- tensor of shape (2,) containing the shape of the image
    Returns:
    boxes -- tensor of shape (None, 4) containing the scaled coordinates of the boxes
    """
    height, width = image_shape
    image_dims = tf.stack([height, width, height, width])
    image_dims = tf.cast(image_dims, tf.float32)
    return boxes * image_dims

In [12]:
def yolo_eval(yolo_outputs, image_shape=(720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates, and classes.
    
    
    """
    
    # Retrieve outputs of the YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Perform Score-filtering with a threshold of score_threshold
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    
    # Scale boxes back to original image shape
    boxes = scale_boxes(boxes, image_shape)

    # Perform Non-max suppression
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
    
    return scores, boxes, classes

In [13]:
yolo_outputs = ( tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                     tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                     tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                     tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
scores, boxes, classes = yolo_eval(yolo_outputs)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

scores[2] = 147.72488
boxes[2] = [2651.31   1417.04   1748.9335 7298.7256]
classes[2] = 74
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


# Test YOLO pre-trained model on images

Defining classes, anchors and image shape

Loading a pre-trained model
